## Imports

In [19]:
from datasets import Dataset
from keras.utils.np_utils import to_categorical

## Constantes

In [20]:
dir = "../../"
dir_data = "data/"
dir_rotulacao = "rotulacao/result/"

arquivo_result = "facebook_1000.json" ## Verificar caso a caso

dir_dataset = "datasets/dataset_facebook/"## Verificar caso a caso
dataset_name = "dataset_concatenate_facebook.json" ## Verificar caso a caso

## Lendo Resultados

In [21]:
resultado_doccano = Dataset.from_json(f"{dir}{dir_data}{dir_rotulacao}{arquivo_result}")

Using custom data configuration default-f83fc1cac197cd4e
Found cached dataset json (/home/thiago/.cache/huggingface/datasets/json/default-f83fc1cac197cd4e/0.0.0)


In [22]:
resultado_doccano

Dataset({
    features: ['id', 'text', 'index', 'link', 'label', 'Comments'],
    num_rows: 1000
})

## Lendo Dataset

In [23]:
dataset = Dataset.from_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Using custom data configuration default-6d109140fc9b5cb9


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/thiago/.cache/huggingface/datasets/json/default-6d109140fc9b5cb9/0.0.0. Subsequent calls will reuse this data.


In [24]:
dataset

Dataset({
    features: ['created_time', 'text', 'permalink_url', 'id_post', 'dataset_origem', 'index', 'repitido', 'labels', 'labels_int'],
    num_rows: 34620
})

## Prepara Dados Resultados

In [25]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'link': Value(dtype='string', id=None),
 'label': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None)}

### Arrumar Colunas de Label

In [26]:
resultado_doccano = resultado_doccano.add_column("labels_int", [None] * len(resultado_doccano))

In [27]:
def arruma_label(data):
    valor = 0
    if "Neutro" in data["label"]:
        valor = 1
    elif "Positivo" in data["label"]:
        valor = 2
    data["label"] = to_categorical(valor, num_classes=3)
    data["labels_int"] = valor
    return data
resultado_doccano = resultado_doccano.map(arruma_label)

Loading cached processed dataset at /home/thiago/.cache/huggingface/datasets/json/default-f83fc1cac197cd4e/0.0.0/cache-d177c14afddaa0fb.arrow


In [28]:
resultado_doccano.features

{'id': Value(dtype='int64', id=None),
 'text': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'link': Value(dtype='string', id=None),
 'label': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'Comments': Sequence(feature=Value(dtype='null', id=None), length=-1, id=None),
 'labels_int': Value(dtype='int64', id=None)}

## Prepara Dados Dataset

In [29]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'repitido': Value(dtype='bool', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None)}

### Adiciona a coluna de "label"

In [30]:
if "labels" not in dataset.features:
    dataset = dataset.add_column("labels", [None] * len(dataset))
    dataset = dataset.add_column("labels_int", [None] * len(dataset))

In [31]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'repitido': Value(dtype='bool', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None)}

## Merge - Dataset & Result Doccano

### Convertendo result Doccano para Dict

In [32]:
dict_resultado_doccano = dict({dado["index"] : [dado["label"], dado["labels_int"]] for dado in resultado_doccano})

### Merge

In [33]:
def merge(data):
    if data["index"] in dict_resultado_doccano.keys():
        data["labels"] = dict_resultado_doccano[data["index"]][0]
        data["labels_int"] = dict_resultado_doccano[data["index"]][1]
    return data
dataset = dataset.map(merge)

  0%|          | 0/34620 [00:00<?, ?ex/s]

## Salvando Dataset

In [34]:
dataset.to_json(f"{dir}{dir_data}{dir_dataset}{dataset_name}")

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

14129028

## Testes

In [35]:
from collections import Counter
print(Counter(dataset["labels_int"]))

Counter({None: 33420, 1.0: 592, 2.0: 577, 0.0: 31})


In [36]:
dataset.features

{'created_time': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None),
 'permalink_url': Value(dtype='string', id=None),
 'id_post': Value(dtype='string', id=None),
 'dataset_origem': Value(dtype='string', id=None),
 'index': Value(dtype='int64', id=None),
 'repitido': Value(dtype='bool', id=None),
 'labels': Sequence(feature=Value(dtype='float64', id=None), length=-1, id=None),
 'labels_int': Value(dtype='float64', id=None)}